In [1]:
import numpy as np

In [20]:
maze_str = '''
XXXXXXXXXXX
X      X TX
X X   T  3X
X X X    2X
XS  X  T 1X
X   X  T  X
XXXXXXXXXXX
'''
maze = [[c for c in line] for line in maze_str.strip().split('\n')]
maze

[['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X'],
 ['X', ' ', ' ', ' ', ' ', ' ', ' ', 'X', ' ', 'T', 'X'],
 ['X', ' ', 'X', ' ', ' ', ' ', 'T', ' ', ' ', '3', 'X'],
 ['X', ' ', 'X', ' ', 'X', ' ', ' ', ' ', ' ', '2', 'X'],
 ['X', 'S', ' ', ' ', 'X', ' ', ' ', 'T', ' ', '1', 'X'],
 ['X', ' ', ' ', ' ', 'X', ' ', ' ', 'T', ' ', ' ', 'X'],
 ['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']]

In [21]:
def find_start_end_position():
    start = None
    ends = []
    for i in range(len(maze)):
        for j in range(len(maze[i])):
            if maze[i][j] == 'S':
                start = (i,j)
            if maze[i][j] in ['1','2','3']:
                ends.append((i,j))
    return start, ends

START, ENDS = find_start_end_position()

In [22]:
ROW = len(maze)
COL = len(maze[0])

In [23]:
# Cache to quicly return possible actions for a given state
state_actions = {}

In [83]:
def get_actions(state):
    '''
    Find the possible action for a given state
    '''
    
    # No action if reach the end
    if maze[state[0]][state[1]] in ['1', '2', '3']:
        return []
    
    if state_actions.get(state):
        return state_actions[state]
    i,j = state
    possible_actions = []
    if i > 1 and maze[i-1][j] != 'X':
        possible_actions.append('UP')
    if i < ROW - 2 and maze[i+1][j] != 'X':
        possible_actions.append('DOWN')
    if j > 1 and maze[i][j-1] != 'X':
        possible_actions.append('LEFT')
    if j < COL - 2 and maze[i][j+1] != 'X':
        possible_actions.append('RIGHT')
    state_actions[state] = possible_actions
    return possible_actions

def pick_action(state, actions, Q_table, epsilon, decay):
    if np.random.random() > epsilon**decay:
        return np.random.choice(actions)
    
    best_action = None
    best_Q_value = np.NINF
    for action in actions:
        Q_value = Q_table.get(state,{}).get(action,0)
        if Q_value > best_Q_value:
            best_action = action
            best_Q_value = Q_value
    return best_action

def take_action(state, action):
    i, j = state
    if action == 'UP':
        return i-1, j
    if action == 'DOWN':
        return i+1, j
    if action == 'LEFT':
        return i, j-1
    if action == 'RIGHT':
        return i, j+1

def reward(state):
    i,j = state
    rewards = {
        '3': 10,
        '2': 3,
        '1': 1,
        'T': -10
    }
    return rewards.get(maze[i][j],-1)


def done_game(state, cache):
    i,j = state
    if maze[i][j] in ['T', '1', '2', '3']:
        cache[maze[i][j]] += 1
        return True
    return False


## Q value update equation
$Q(s,a) = Q(s,a) + \alpha \cdot( R(s') + \gamma \cdot Max(Q(s',a')) - Q(s,a))$

where 

$Q(s,a)$: Q value at state s and take action `a`

$R(s')$: The reward after take action `a` from state `s` and move to state `s'`

$Max(Q(s',a'))$: The maximum reward at state `s'` and take action `a'` for all `a'`<br>

$\alpha$: Learning rate

$\gamma$: Discount factor


In [84]:
def find_and_insert_Q_value(state, action, Q_table):
    if Q_table.get(state) is None:
        Q_table[state] = {}
    if Q_table[state].get(action) is None:
        Q_table[state][action] = 0
    return Q_table[state][action]

def get_max_Q_value(state,actions,Q_table):
    if not actions:
        return 0
    
    max_Q = np.NINF
    for action in actions:
        val = find_and_insert_Q_value(state, action, Q_table)
        if val > max_Q:
            max_Q = val
    return max_Q

def update_Q_table(state, action, Q_table, gamma, lr):    
    value = find_and_insert_Q_value(state, action, Q_table)
    
    next_state = take_action(state, action)
    next_state_actions = get_actions(next_state)
    # Q learning update function
    value = value + lr * (reward(next_state) + gamma * get_max_Q_value(next_state, next_state_actions, Q_table) - value)
    Q_table[state][action] = value
    
    return next_state

In [85]:
DISCOUNT = 0.9
LEARNING_RATE = 0.1
EPSILON = 0.95
DECAY = 30

In [86]:
Q_table = {}

In [87]:
game_end_state = {
    '3': 0,
    '2': 0,
    '1': 0,
    'T': 0,
    'STOP': 0, 
}

In [88]:
game = 0
MAX_STEP = 100
while game < 1000:
    current_pos = START
    step = 0
    while not done_game(current_pos, game_end_state) and step < MAX_STEP:
        actions = get_actions(current_pos)
        action = pick_action(current_pos, actions, Q_table, EPSILON, DECAY)
        current_pos = update_Q_table(current_pos, action, Q_table, DISCOUNT, LEARNING_RATE)
        step+=1
    
    if step >= MAX_STEP:
        game_end_state['STOP']+=1
    
    if game % 5 == 0:
        DECAY -= 1
        DECAY = max(1, DECAY)
    
    game += 1

In [89]:
game_end_state

{'3': 245, '2': 585, '1': 23, 'T': 143, 'STOP': 4}

In [90]:
maze = [[c for c in line] for line in maze_str.strip().split('\n')]
maze

[['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X'],
 ['X', ' ', ' ', ' ', ' ', ' ', ' ', 'X', ' ', 'T', 'X'],
 ['X', ' ', 'X', ' ', ' ', ' ', 'T', ' ', ' ', '3', 'X'],
 ['X', ' ', 'X', ' ', 'X', ' ', ' ', ' ', ' ', '2', 'X'],
 ['X', 'S', ' ', ' ', 'X', ' ', ' ', 'T', ' ', '1', 'X'],
 ['X', ' ', ' ', ' ', 'X', ' ', ' ', 'T', ' ', ' ', 'X'],
 ['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']]

In [91]:
current_pos = START
while not done_game(current_pos,game_end_state):
    if not current_pos == START:
        maze[current_pos[0]][current_pos[1]] = '*'
    actions = get_actions(current_pos)
    action = pick_action(current_pos, actions, Q_table, 0, 0)
    print(current_pos, action)
    current_pos = take_action(current_pos, action)
print(f'Stop at {maze[current_pos[0]][current_pos[1]]}')

(4, 1) RIGHT
(4, 2) RIGHT
(4, 3) UP
(3, 3) UP
(2, 3) RIGHT
(2, 4) RIGHT
(2, 5) DOWN
(3, 5) RIGHT
(3, 6) RIGHT
(3, 7) RIGHT
(3, 8) UP
(2, 8) RIGHT
Stop at 3


In [92]:
maze

[['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X'],
 ['X', ' ', ' ', ' ', ' ', ' ', ' ', 'X', ' ', 'T', 'X'],
 ['X', ' ', 'X', '*', '*', '*', 'T', ' ', '*', '3', 'X'],
 ['X', ' ', 'X', '*', 'X', '*', '*', '*', '*', '2', 'X'],
 ['X', 'S', '*', '*', 'X', ' ', ' ', 'T', ' ', '1', 'X'],
 ['X', ' ', ' ', ' ', 'X', ' ', ' ', 'T', ' ', ' ', 'X'],
 ['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']]